In [1]:
import jsonlines
from dialop.games import OptimizationGame
from dialop.games.optimization import WORKERS, TASKS

with jsonlines.open('data/optimization.jsonl') as jsonl_f:
     data = [obj for obj in jsonl_f]


In [2]:
import random
idx = random.randint(0, len(data))

def stringify_proposal(assignment):
    assignment.sort(key=lambda x: x[1])
    out_str = ""
    out_str = '\n'.join([f"{TASKS[a[1]]}: {WORKERS[a[0]]}" for a in assignment])
    return out_str

d = data[idx]

game = OptimizationGame.create_from_game_state(d)

table = game.tables[0]
obs_string = "Player 1 knowledge: " 
for row in table[1:]:
    obs_string += row[0] + ": "
    obs_string += ', '.join([f"{c} for {TASKS[i]}" for i, c in enumerate(row[1:]) if type(c) == int])
    obs_string += " \n "
print(obs_string)

table = game.tables[1]
obs_string = "Player 2 knowledge: " 
for row in table[1:]:
    obs_string += row[0] + ": "
    obs_string += ', '.join([f"{TASKS[i]}:{c}" for i, c in enumerate(row[1:]) if type(c) == int])
    obs_string += " \n "
print(obs_string)

print(f"{'-'*50} CONVERSATION {'-'*50}")
final_proposal = None
for a in d['action_log']:
    output_str = f"Player {a['player']+1}: "
    if a['type'] == 'message':
        output_str += a['message']['data']
    elif a['type'] == 'proposal':
        output_str += a['proposal'].replace('<br/>&emsp;', '\n')
        final_proposal = a['proposal_ids']
    elif a['type'] == 'proposal_response':
        output_str += 'ACCEPT' if a['response']['accept'] else 'REJECT'
    print(output_str)

print("-"*100)
best_assignment, best_reward = game.table.find_max_value_known_assignment([d['mask1'], d['mask2']])
print("\nFINAL PROPOSAL:")
print(stringify_proposal(final_proposal))
print(f"Proposal reward = {d['proposal_reward']}")
print("\nOPTIMAL ASSIGNMENT:")
print(stringify_proposal(best_assignment))
print(f"Best possible reward = {d['best_assignment_reward']}")